In [6]:
import pdat
import os
import numpy as np
np.bool = bool
import antropy as ant
import matplotlib.pyplot as plt
import psrchive
import nolds 


In [2]:
file_names = ['f210820_041554.DFp', 
            'f210820_105959.DFp',
            'f210823_120959.DFp',
            'f210823_124534.DFp',
            'f210828_190858.DFp',
            'f210906_152306.DFp',
            'f210920_142859.DFp',
            'f210921_002549.DFp',
            'f210921_004724.DFp',
            'f210922_085559.DFp',
            'f210929_090945.DFp']

In [ ]:
# psrf1 = pdat.psrfits('Wang/raw/f210820_041554.DFp')
# psrf1[4].read_header()

for address in file_names:   
    # print('Wang/'+address)
    psrfit_data = pdat.psrfits('Wang/single_pulse_DFp/'+address)
    # if address=="f210906_152306.DFp" or address == 'f210921_004724.DFp':
        # data_all = psrfit_data[1].read()
    # else:
    data_all = psrfit_data[2].read()
    PSR_name = data_all[0]
    # Extract the first element of the tuple and decode it
    decoded_name = PSR_name[0].decode('utf-8').strip()
    PSR_name = decoded_name
    # Extract the actual pulsar name (e.g., "J1012+5307")
    pulsar_name = decoded_name.split()[1]  # Split by whitespace and take the second part
    print(PSR_name)
    del data_all
    data_all = psrfit_data[4].read()
      
    DATA     = data_all["DATA"] 
    DAT_SCL  = data_all["DAT_SCL"]
    DAT_OFFS = data_all["DAT_OFFS"]
    DAT_FREQ = data_all["DAT_FREQ"]
    print(np.shape(DATA))
    Flux_all = np.zeros([np.shape(DATA)[3],1])  # average pulse profile
    
    # initilizing Statistical measures for single pulses 
    perm_entropy_single_pulses = np.zeros([np.shape(DATA)[0],1])
    fractal_dims_single_pulses = np.zeros([np.shape(DATA)[0],1])
    
    # Loading Single Pulses 
    for i in range(np.shape(DATA)[0]):
        Single_Pulses = np.zeros(np.shape(DATA)[3]) # Single pulse profiles for each frequesncy 
        Single_Pulses = DATA[i,0,0,:]*DAT_SCL[i] + DAT_OFFS[i]
        # call any statistics/complexity functions here 
        perm_entropy_single_pulses[i] = ant.perm_entropy(Single_Pulses, delay=[1,2,3,4], order=10, normalize=True)
        fractal_dims_single_pulses[i] = nolds.corr_dim(Single_Pulses, emb_dim==10)
        
    
    # Plotting the histogram of the permutation entropy values
    p = plt.hist(perm_entropy_single_pulses,
                    # bins=np.linspace(0.9, 1, 100),
                    density=True,
                    alpha=0.75)
    plt.xlabel('Fractal Dimension')
    plt.ylabel('Probability Density')
    # plt.xlim([0.9, 1])
    # plt.ylim([0, 1])
    plt.title(PSR_name)
    plt.savefig('statFigs/fractal_dim/fractal_dim-kmax-10/' + address.replace(".DFp", "")+'.png', dpi=400)
    plt.show()
    #plotting average pulse 
    for i in range(np.shape(DATA)[3]):
        Flux_all[i] = np.sum(DATA[:,0,0,i]*DAT_SCL[:] + DAT_OFFS[:])

    # perm_entropy_average_pulse = ant.perm_entropy(Flux_all, order=3, delay=1, normalize=True)
    # print(perm_entropy_average_pulse)

    
    # # # seems like we have 512 time channels in 
    # fig = plt.figure()
    # plt.plot(Flux_all/np.max(Flux_all))
    # plt.xlabel('Phase')
    # plt.ylabel('Normalized Total Flux')
    # plt.savefig('PulseProfiles/' + address+'.png', dpi=400)
    # plt.close()

Loading PSRFITS file from path:
    'Wang/single_pulse_DFp/f210820_041554.DFp'.
PSRJ            J1012+5307
(101769, 1, 1, 512)


AttributeError: module 'nolds' has no attribute 'hfd'

In [21]:
import psrchive



In [22]:
arch  = psrchive.Archive_load('Vela/single_pulses/2021-05-10-070130.ar')

In [23]:
arch.get_source()

'J0835-4510'

In [24]:
arch.get_data()

array([[[[ 1.54377532e+00,  1.33593500e+00,  1.39990222e+00, ...,
           1.46227300e+00,  1.25174522e+00,  1.44859612e+00],
         [ 4.55890369e+00,  5.87633276e+00,  5.45102215e+00, ...,
           3.98534513e+00,  5.35647774e+00,  3.04421711e+00],
         [ 8.76651478e+00,  8.81715202e+00,  9.46782112e+00, ...,
           6.49338055e+00,  6.91879988e+00,  9.22002983e+00],
         ...,
         [ 8.78921700e+00,  7.59977150e+00,  7.23789597e+00, ...,
           7.77467155e+00,  7.02701998e+00,  7.80529451e+00],
         [ 4.28958750e+00,  6.19564152e+00,  4.72500134e+00, ...,
           5.90263653e+00,  7.15794992e+00,  5.94471216e+00],
         [ 1.30236542e+00,  1.34560204e+00,  1.78218853e+00, ...,
           1.64349258e+00,  1.63129103e+00,  1.36564136e+00]],

        [[ 1.51859212e+00,  1.47786355e+00,  1.25458193e+00, ...,
           1.37864017e+00,  1.53488350e+00,  1.26759624e+00],
         [ 4.62819290e+00,  3.99195099e+00,  4.74190807e+00, ...,
           4.37831783e